Q1) Consider the following Dataset containing information regarding alcohol consumption in different countries. 
An analyst performs different operations to carry out certain tasks to help the analyst in performing the task.

 Which continent drinks more Wine on average? [2 marks]

 For each continent print the statistics for Beer consumption. [2 mark]
 
 Print the median alcohol consumption per continent for every column. [1 marks]

In [ ]:
import pandas as pd
df1 = pd.read_csv('drinks.csv')
df1_copy = df1 
#df1_copy.head()

#1st ques code :

x = df1_copy.groupby('continent').agg(
    {
        'wine_servings' : 'mean'
    }
)
print(x.sort_values('wine_servings'))
print("\n")

#2nd ques code: 
y = df1_copy.groupby('continent').agg(
    {
        'beer_servings' : 'describe'
    }
)
print(y)
print("\n")

#3rd ques code :

z = df1_copy.groupby('continent').median()
print(z)

Q2) Consider a Euro 2012 football tournament.


Football analysts decide certain criteria in order to decide which team has a good chance to win this cup. Your task is to help those analysts by performing some tasks given below.

 Find the teams who are participating in this tournament. [1 marks]

 Find the shooting accuracy of all the teams that have scored more than 5 goals. [1 marks]

 Find the total number of goals scored in the tournament. [1 marks]

 Sort the teams on the basis of goals scored(non-ascending order) [1 marks]
 
 Find the team with the most number of red cards. [1 marks]

In [ ]:
import pandas as pd
df2 = pd.read_csv('Euro_2012_stats_TEAM.csv')
# df2
#code for 1st ques :
print(df2.Team)
print(df2.shape[0])
print("\n")
print("---------------------------------------------------")

#code for 2nd ques :

accuracy = df2.loc[df2["Goals"]>5,["Team","Shooting Accuracy"]]
print(accuracy)
print("\n")
print("----------------------------------------------------")

#code for 3rd ques :

print(df2["Goals"].sum())
print("\n")
print("-------------------------------------------------------")

#code for 4th ques :

sortedDf = df2.loc[:,["Team","Goals"]]
print(sortedDf.sort_values("Goals",ascending = False))
print("\n")
print("--------------------------------------------------------")


#code for 5th ques :

redCards = df2.loc[:,["Team","Red Cards"]]
print(redCards[redCards["Red Cards"]==redCards["Red Cards"].max()])
print("\n")
print("---------------------------------------------------------")



Q3) Consider the given breast cancer dataset from the given google drive link, the last column has the target variable 


(2 classes: "B" for benign and "M" for malignant) and the remaining columns have the features.


Implement the decision tree algorithm from scratch. You can use the Gini index to decide on the splitting attribute on the breast cancer dataset. Make use of only NumPy, pandas. Calculate the accuracy of the decision tree. [10 marks]


Data set info : (the second column is the class attribute).

In [ ]:
import numpy as np
import pandas as pd
df3 = pd.read_csv('wdbc.data')
#print(df3)
# df3.head(10)
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision nodes :
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf nodes :
        self.value = value


# The following would be the code for tree class :

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


# The next few lines of code split the data set :

X = df3.iloc[:, :-1].values
Y = df3.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

# The next few lines of code are for fitting the model :

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

# The last part of the question - calculating the accuracy score : we use sklearn.matrix module for that as it is allowed :

Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

